Link to article explainig code:
<url>https://cnvrg.io/pytorch-lstm/</url>

In [1]:
# imports
import torch as torch
import torch.nn as nn
from torch.autograd import Variable 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# read processed data
df = pd.read_csv('../../dataframes/monthly_processed.csv', index_col='date', parse_dates=True)

In [3]:
# variable definitions
num_epochs = 1                  # number of epochs
learning_rate = 0.001           # 0.001 lr

seq_length = 36                 # sequence length of sequence given to the LSTM layer
pred_length = 12                # prediction length of output sequence produced by LSTM layer 
                                # (multiples of twelve since there is yearly seasonality)

input_size = len(df.columns)    # number of features (amount of input variables)
output_size = 1                 # size of output (peru fish oil price so just one variable)
hidden_size = 5                 # number of features in hidden state (this number should be treated as a hyperparameter)
num_layers = 1                  # number of stacked lstm layers

num_classes = pred_length       # number of output classes 

In [4]:
# function to create list of sequences of length seq_length
def create_series(data):
    # the amount of time-steps for which we have data
    total_timesteps = len(data)

    # arrays that hold the input sequences and the corresponding target sequences respectively.
    sequencesX, sequencesY = [], []
    if (total_timesteps < seq_length + pred_length):
        # verify that enough time-steps of the data are available to create input sequence with their corresponding targets.
        raise Exception("This dataframe cannot be used to create sequences of length " + str(seq_length + pred_length))
    
    # create all possible sequences of length seq_length and their 
    for i in range(total_timesteps - seq_length - pred_length):
        # append the input sequence
        sequencesX.append(data.iloc[i:(i+seq_length), :])
        # append the corresponding (target) output sequence
        sequencesY.append(data.iloc[(i+seq_length):(i+seq_length+pred_length), 0:1])
    
    # return the sequences
    return np.array(sequencesX), np.array(sequencesY)

In the following code block the X and y sets are created.

The shape of `X` is as follows:<br>
The element `X[s][i][j]` contains the value of the $j$'th input variable (out of the `len(df.columns)` different input variables) at the $i$'th time-step (out of the `seq_length` time-steps in the sequence) for the $s$'th sequence in the data set `X`.<br>

The shape of `y` is as follows:<br>
The element `y[s][i][0]` contains the value of the peru fish oil price (target) at the $i$'th time-step (out of the `pred_length` time-steps in the sequence) for the $s$'th sequence in the data set `y`.<br>
The last zero-index is used because all the values are wrapped in length one arrays.


In [5]:
# split data into train and test sets
# TODO: May also need a validation set to determine optimal hyperparameters
df_train = df[(df.index > "1996-12-01") & (df.index < "2010-01-01")]    # use only past 1997 due to geopolitical changes
df_test = df[df.index >= "2010-01-01"]

# create the sequences to train on
X_train, y_train = create_series(df_train)

# create the sequences to test on
X_test, y_test = create_series(df_test)

# print shapes
print("Training input/output shapes respectively:", X_train.shape, y_train.shape)
print("Testing input/output shapes respectively:", X_test.shape, y_test.shape) 

Training input/output shapes respectively: (108, 36, 7) (108, 12, 1)
Testing input/output shapes respectively: (102, 36, 7) (102, 12, 1)


The datasets are converted to PyTorch tensors and the shapes are printed.<br>
The input shape is of the form `(nr_of_sequences, seq_length, input_size)`.<br>
The output shape is of the form `(nr_of_sequences, pred_length, output_size)`.

If this needs to be reshaped the `torch.reshape(data, shape)` function can be used.

In [6]:
# convert data to pytorch variables
X_train_tensors = Variable(torch.Tensor(X_train))
y_train_tensors = Variable(torch.Tensor(y_train))
X_test_tensors = Variable(torch.Tensor(X_test))
y_test_tensors = Variable(torch.Tensor(y_test)) 

# print shapes
print("Training input/output shapes respectively:", X_train_tensors.shape, X_train_tensors.shape)
print("Testing input/output shapes respectively:", X_test_tensors.shape, y_test_tensors.shape) 

Training input/output shapes respectively: torch.Size([108, 36, 7]) torch.Size([108, 36, 7])
Testing input/output shapes respectively: torch.Size([102, 36, 7]) torch.Size([102, 12, 1])


In [7]:
torch.cat((torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]),torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])), 0)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8])

The input size of the first fully connected layer is equal to `(hidden_size * seq_length)` because the LSTM outputs the hidden state (containing `hidden_size` variables) for every time-step in the input which contains `seq_length` time-steps. The output size of this layer is a hyperparameter for which a default of twice the amount of inputs is used.

The lstm layer can also use an initial hidden state and initial cell state for each time-step in the sequence of `seq_length` but since these are not provided it defaults to zero.<br>
See the input section of the <a url=https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html>documentation</a> for more information.

The output of the lstm layer is `output, (output_hidden_states, output_cell_states)` where `output` contains for every sequence in the provided `input` the hidden state at each time-step.<br>
Furthermore, `output_hidden_states` and `output_cell_states` contain for every sequence the hidden state and cell state of the last time-step, respectively.

In [8]:
# definition of LSTM model
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes      # number of classes
        self.num_layers = num_layers        # number of layers
        self.input_size = input_size        # input size
        self.hidden_size = hidden_size      # hidden state

        fc_1_input_size = hidden_size * seq_length
        fc_1_output_size = hidden_size * seq_length * 2     # output size of first fully connected layer (hyperparameter)

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) # lstm
        self.fc_1 = nn.Linear(fc_1_input_size, fc_1_output_size) # first fully connected layer
        self.fc_2 = nn.Linear(fc_1_output_size, num_classes) # second fully connected layer

        self.relu = nn.ReLU()
    
    def forward(self, input):
        # propagate input through LSTM
        output, (output_hidden_states, output_cell_states) = self.lstm(input) # retrieve output from lstm by providing input (the initial hidden and cell states can be set but default sets to zero)
        # flattened_output_hidden_states = torch.flatten(output_hidden_states) # flattened output hidden states
        # flattened_output_cell_states = torch.flatten(output_cell_states) # flattened output cell states
        # flattened_output = torch.cat((flattened_output_hidden_states, flattened_output_cell_states), 0) # concatenate the flattened output hidden and cell states (among dimension 0 since they only have one)
        flattened_output = torch.flatten(output) # flattened output
        out = self.relu(flattened_output) # relu
        out = self.fc_1(out) # first fully connected layer
        out = self.relu(out) # relu
        out = self.fc_2(out) # second fully connected layer
        return out

In [9]:
# create the lstm layer 
lstm = LSTM(num_classes, input_size, hidden_size, num_layers) # lstm layer

In [10]:
# define the loss function and weight updating method
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 

In [11]:
lstm.forward(X_train_tensors[50])

1

In [12]:
# train the model
for epoch in range(num_epochs):
  # using batches of size 1
  for sequence in range(X_train_tensors.size(0)):
    outputs = lstm.forward(X_train_tensors[sequence]) # forward pass
    optimizer.zero_grad() # calculate the gradient, manually setting to 0
  
    # obtain the loss function
    loss = criterion(outputs, y_train_tensors[sequence])
  
    loss.backward() # calculates the loss of the loss function
  
    optimizer.step() # improve from loss, i.e backprop
  if epoch % 10 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

C:\Users\seppe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([12, 1])) that is different to the input size (torch.Size([12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 0.05533


In [13]:
# make predictions for the test set

# compute loss on test set

# plot test set predictions